## Integrantes do grupo:

**(Nome do aluno) - (R.A)**

Luiz Fernando Ferreira Berber - 817123421

Lucas Alves de Sousa - 817116864

Matheus Henrique Silva Paiva - 817123241 

Mateus Sampaio de Souza - 817123420

Pedro Jacó Cunha Mesquita - 817124121

In [52]:
!pip install numpy
!pip install pandas
!pip install mysql-connector-python

In [77]:
import numpy as np
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [78]:
def create_connection(host_name, user_name, user_password):
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [79]:
def execute_query(connection, query):
    cursor = connection.cursor()
    
    try:
        cursor.execute(query)
        connection.commit()
        
        print("Query executed successfully")
        
        if query.lstrip().upper().startswith('INSERT'):
            print('Last insert id:', cursor.lastrowid)
    except Error as e:
        print(f"The error '{e}' occurred")

In [134]:
def insert_dimensao_orgao():
    cod_orgao_superior = dados_despesa['Código Órgão Superior']
    nome_orgao_superior = dados_despesa['Nome Órgão Superior']

    cod_orgao_subordinado = dados_despesa['Código Órgão Subordinado']
    nome_orgao_subordinado = dados_despesa['Nome Órgão Subordinado']

    cod_unidade_orcamentaria = dados_despesa['Código Unidade Orçamentária']
    nome_unidade_orcamentaria = dados_despesa['Nome Unidade Orçamentária']

    for i in range(cod_orgao_superior.size):
        insert_dimensao_orgao = f"""
        INSERT INTO orcamento_publico.TD_ORGAO
        (
            COD_ORGAO_SUPERIOR
        ,	NOME_ORGAO_SUPERIOR
        ,	COD_ORGAO_SUBORDINADO
        ,	NOME_ORGAO_SUBORDINADO
        ,	COD_UNIDADE_ORCAMENTARIA
        ,	NOME_UNIDADE_ORCAMENTARIA
        ,	DATA_CRIACAO
        )
        VALUES
        (
            {cod_orgao_superior[i]}
        ,   '{nome_orgao_superior[i]}'
        ,   {cod_orgao_subordinado[i]}
        ,   '{nome_orgao_subordinado[i]}'
        ,   {cod_unidade_orcamentaria[i]}
        ,   '{nome_unidade_orcamentaria[i]}'
        ,   NOW()
        );"""

        execute_query(connection, insert_dimensao_orgao)

In [148]:
def insert_dimensao_programa():
    cod_programa_ocamentario = dados_despesa['Código Programa Orçamentário']
    nome_programa_orcamentario = dados_despesa['Nome Programa Orçamentário']
    cod_acao = dados_despesa['Código Ação']
    nome_acao = dados_despesa['Nome Ação']
    
    for i in range(cod_programa_ocamentario.size):
        insert_dimensao_programa = f"""
        INSERT INTO orcamento_publico.TD_PROGRAMA
        (
            COD_PROGRAMA_ORCAMENTARIO
        ,	NOME_PROGRAMA_ORCAMENTARIO
        ,	COD_ACAO
        ,	NOME_ACAO
        ,	DATA_CRIACAO
        )
        VALUES
        (
         {cod_programa_ocamentario[i]}
        ,'{nome_programa_orcamentario[i]}'
        ,{cod_acao[i]}
        ,'{nome_acao[i]}'
        ,NOW()
        );
        """
        
        execute_query(connection, insert_dimensao_programa)

In [133]:
def insert_dimensao_data():
    insert_dimensao_data = f"""
    INSERT INTO orcamento_publico.TD_DATA
    (
        MES
    ,	ANO
    ,	DATA_CRIACAO
    )
    VALUES
    (
        01
    ,   2019
    ,   NOW()
    );
    """
    
    execute_query(connection, insert_dimensao_data)

In [86]:
mysql_host_name = '127.0.0.1'
mysql_user_name = 'mateus'
mysql_password = 'root123'

In [87]:
filepath = '../Datas'

In [88]:
dados_despesa = pd.read_csv(f'{filepath}/Despesas/201901_Despesas.zip', sep=';', encoding='mbcs')

In [135]:
insert_dimensao_data()
insert_dimensao_orgao()
insert_dimensao_programa()

Query executed successfully
Last insert id: 1
Query executed successfully
Last insert id: 1
Query executed successfully
Last insert id: 2
Query executed successfully
Last insert id: 3
Query executed successfully
Last insert id: 4
Query executed successfully
Last insert id: 5
Query executed successfully
Last insert id: 6
Query executed successfully
Last insert id: 7
Query executed successfully
Last insert id: 8
Query executed successfully
Last insert id: 9
Query executed successfully
Last insert id: 10
Query executed successfully
Last insert id: 11
Query executed successfully
Last insert id: 12
Query executed successfully
Last insert id: 13
Query executed successfully
Last insert id: 14
Query executed successfully
Last insert id: 15
Query executed successfully
Last insert id: 16
Query executed successfully
Last insert id: 17
Query executed successfully
Last insert id: 18
Query executed successfully
Last insert id: 19
Query executed successfully
Last insert id: 20
Query executed successf

Query executed successfully
Last insert id: 203
Query executed successfully
Last insert id: 204
Query executed successfully
Last insert id: 205
Query executed successfully
Last insert id: 206
Query executed successfully
Last insert id: 207
Query executed successfully
Last insert id: 208
Query executed successfully
Last insert id: 209
Query executed successfully
Last insert id: 210
Query executed successfully
Last insert id: 211
Query executed successfully
Last insert id: 212
Query executed successfully
Last insert id: 213
Query executed successfully
Last insert id: 214
Query executed successfully
Last insert id: 215
Query executed successfully
Last insert id: 216
Query executed successfully
Last insert id: 217
Query executed successfully
Last insert id: 218
Query executed successfully
Last insert id: 219
Query executed successfully
Last insert id: 220
Query executed successfully
Last insert id: 221
Query executed successfully
Last insert id: 222
Query executed successfully
Last insert 

Query executed successfully
Last insert id: 376
Query executed successfully
Last insert id: 377
Query executed successfully
Last insert id: 378
Query executed successfully
Last insert id: 379
Query executed successfully
Last insert id: 380
Query executed successfully
Last insert id: 381
Query executed successfully
Last insert id: 382
Query executed successfully
Last insert id: 383
Query executed successfully
Last insert id: 384
Query executed successfully
Last insert id: 385
Query executed successfully
Last insert id: 386
Query executed successfully
Last insert id: 387
Query executed successfully
Last insert id: 388
Query executed successfully
Last insert id: 389
Query executed successfully
Last insert id: 390
Query executed successfully
Last insert id: 391
Query executed successfully
Last insert id: 392
Query executed successfully
Last insert id: 393
Query executed successfully
Last insert id: 394
Query executed successfully
Last insert id: 395
Query executed successfully
Last insert 

Query executed successfully
Last insert id: 589
Query executed successfully
Last insert id: 590
Query executed successfully
Last insert id: 591
Query executed successfully
Last insert id: 592
Query executed successfully
Last insert id: 593
Query executed successfully
Last insert id: 594
Query executed successfully
Last insert id: 595
Query executed successfully
Last insert id: 596
Query executed successfully
Last insert id: 597
Query executed successfully
Last insert id: 598
Query executed successfully
Last insert id: 599
Query executed successfully
Last insert id: 600
Query executed successfully
Last insert id: 601
Query executed successfully
Last insert id: 602
Query executed successfully
Last insert id: 603
Query executed successfully
Last insert id: 604
Query executed successfully
Last insert id: 605
Query executed successfully
Last insert id: 606
Query executed successfully
Last insert id: 607
Query executed successfully
Last insert id: 608
Query executed successfully
Last insert 

Query executed successfully
Last insert id: 806
Query executed successfully
Last insert id: 807
Query executed successfully
Last insert id: 808
Query executed successfully
Last insert id: 809
Query executed successfully
Last insert id: 810
Query executed successfully
Last insert id: 811
Query executed successfully
Last insert id: 812
Query executed successfully
Last insert id: 813
Query executed successfully
Last insert id: 814
Query executed successfully
Last insert id: 815
Query executed successfully
Last insert id: 816
Query executed successfully
Last insert id: 817
Query executed successfully
Last insert id: 818
Query executed successfully
Last insert id: 819
Query executed successfully
Last insert id: 820
Query executed successfully
Last insert id: 821
Query executed successfully
Last insert id: 822
Query executed successfully
Last insert id: 823
Query executed successfully
Last insert id: 824
Query executed successfully
Last insert id: 825
Query executed successfully
Last insert 

...

Query executed successfully
Last insert id: 41858
Query executed successfully
Last insert id: 41859
Query executed successfully
Last insert id: 41860
Query executed successfully
Last insert id: 41861
Query executed successfully
Last insert id: 41862
Query executed successfully
Last insert id: 41863
Query executed successfully
Last insert id: 41864
Query executed successfully
Last insert id: 41865
Query executed successfully
Last insert id: 41866
Query executed successfully
Last insert id: 41867
Query executed successfully
Last insert id: 41868
Query executed successfully
Last insert id: 41869
Query executed successfully
Last insert id: 41870
Query executed successfully
Last insert id: 41871
Query executed successfully
Last insert id: 41872
Query executed successfully
Last insert id: 41873
Query executed successfully
Last insert id: 41874
Query executed successfully
Last insert id: 41875
Query executed successfully
Last insert id: 41876
Query executed successfully
Last insert id: 41877


Query executed successfully
Last insert id: 42068
Query executed successfully
Last insert id: 42069
Query executed successfully
Last insert id: 42070
Query executed successfully
Last insert id: 42071
Query executed successfully
Last insert id: 42072
Query executed successfully
Last insert id: 42073
Query executed successfully
Last insert id: 42074
Query executed successfully
Last insert id: 42075
Query executed successfully
Last insert id: 42076
Query executed successfully
Last insert id: 42077
Query executed successfully
Last insert id: 42078
Query executed successfully
Last insert id: 42079
Query executed successfully
Last insert id: 42080
Query executed successfully
Last insert id: 42081
Query executed successfully
Last insert id: 42082
Query executed successfully
Last insert id: 42083
Query executed successfully
Last insert id: 42084
Query executed successfully
Last insert id: 42085
Query executed successfully
Last insert id: 42086
Query executed successfully
Last insert id: 42087


Query executed successfully
Last insert id: 42272
Query executed successfully
Last insert id: 42273
Query executed successfully
Last insert id: 42274
Query executed successfully
Last insert id: 42275
Query executed successfully
Last insert id: 42276
Query executed successfully
Last insert id: 42277
Query executed successfully
Last insert id: 42278
Query executed successfully
Last insert id: 42279
Query executed successfully
Last insert id: 42280
Query executed successfully
Last insert id: 42281
Query executed successfully
Last insert id: 42282
Query executed successfully
Last insert id: 42283
Query executed successfully
Last insert id: 42284
Query executed successfully
Last insert id: 42285
Query executed successfully
Last insert id: 42286
Query executed successfully
Last insert id: 42287
Query executed successfully
Last insert id: 42288
Query executed successfully
Last insert id: 42289
Query executed successfully
Last insert id: 42290
Query executed successfully
Last insert id: 42291


Query executed successfully
Last insert id: 42479
Query executed successfully
Last insert id: 42480
Query executed successfully
Last insert id: 42481
Query executed successfully
Last insert id: 42482
Query executed successfully
Last insert id: 42483
Query executed successfully
Last insert id: 42484
Query executed successfully
Last insert id: 42485
Query executed successfully
Last insert id: 42486
Query executed successfully
Last insert id: 42487
Query executed successfully
Last insert id: 42488
Query executed successfully
Last insert id: 42489
Query executed successfully
Last insert id: 42490
Query executed successfully
Last insert id: 42491
Query executed successfully
Last insert id: 42492
Query executed successfully
Last insert id: 42493
Query executed successfully
Last insert id: 42494
Query executed successfully
Last insert id: 42495
Query executed successfully
Last insert id: 42496
Query executed successfully
Last insert id: 42497
Query executed successfully
Last insert id: 42498


Query executed successfully
Last insert id: 42650
Query executed successfully
Last insert id: 42651
Query executed successfully
Last insert id: 42652
Query executed successfully
Last insert id: 42653
Query executed successfully
Last insert id: 42654
Query executed successfully
Last insert id: 42655
Query executed successfully
Last insert id: 42656
Query executed successfully
Last insert id: 42657
Query executed successfully
Last insert id: 42658
Query executed successfully
Last insert id: 42659
Query executed successfully
Last insert id: 42660
Query executed successfully
Last insert id: 42661
Query executed successfully
Last insert id: 42662
Query executed successfully
Last insert id: 42663
Query executed successfully
Last insert id: 42664
Query executed successfully
Last insert id: 42665
Query executed successfully
Last insert id: 42666
Query executed successfully
Last insert id: 42667
Query executed successfully
Last insert id: 42668
Query executed successfully
Last insert id: 42669


dtype('int64')